# DS-поток, весна 2025
## Задание ADL.3
### LLM, Alignment.

**Правила:**

* Дедлайны см. в боте. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Выполненную работу нужно отправить телеграм-боту `@miptstats_ds24_bot`. Для начала работы с ботом каждый раз отправляйте `/start`. Дождитесь подтверждения от бота, что он принял файл. Если подтверждения нет, то что-то не так. **Работы, присланные иным способом, не принимаются.**
* Дедлайны см. в боте. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Прислать нужно **ноутбук в формате `ipynb`**.
* Следите за размером файлов. **Бот не может принимать файлы весом более 20 Мб.** Если файл получается больше, заранее разделите его на несколько.
* Выполнять задание необходимо полностью самостоятельно. **При обнаружении списывания все участники списывания получат штраф.**
* Решения, размещенные на каких-либо интернет-ресурсах, не принимаются. Кроме того, публикация решения в открытом доступе может быть приравнена к предоставлении возможности списать.
* Для выполнения задания используйте этот ноутбук в качестве основы, ничего не удаляя из него. Можно добавлять необходимое количество ячеек.
* Комментарии к решению пишите в markdown-ячейках.
* Выполнение задания (ход решения, выводы и пр.) должно быть осуществлено на русском языке.
* Если код будет не понятен проверяющему, оценка может быть снижена.
* Никакой код из данного задания при проверке запускаться не будет. *Если код студента не выполнен, недописан и т.д., то он не оценивается.*
* В каждой задаче не забывайте делать **пояснения и выводы**.
* **Код из рассказанных на занятиях ноутбуков** можно использовать без ограничений.

**Баллы за задание:**

* Задача 1 &mdash; 40 баллов;
* Задача 2 &mdash; 60 баллов.

In [ ]:
%%capture
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

In [ ]:
import json

import torch
import unsloth

from trl import (
    SFTTrainer,
    DataCollatorForCompletionOnlyLM
)

from datasets import Dataset, load_dataset
from transformers import (
    TrainingArguments,
    AutoModelForCausalLM,
    AutoTokenizer, 
    LlamaTokenizerFast
)
from unsloth import (
    FastLanguageModel,
    is_bfloat16_supported
)
from unsloth.chat_templates import get_chat_template

---
### Задача 1. 

В этой задаче вам предложено провести сравнительное исследование, оценив эффективность нескольких LLM в различных форматах: zero-/few-shot для задачи анализа тональности текста на датасете отзывов IMDB.

---
**Форматы zero-/few-shot**

Современные большие языковые модели (LLM), обученные следовать инструкциям, позволяют решать задачи классификации, если правильно оформить запрос к ним. Подготовку надежной инструкции для решения задачи иногда еще называют промптингом. Подробнее можно почитать <a href="https://docs.mistral.ai/guides/prompting_capabilities/" target="_top">здесь</a> или <a href="https://huggingface.co/docs/transformers//tasks/prompting" target="_top">здесь</a>, или самостоятельно обсудить с LLM!


Рассмотрим два довольно популярных подхода.
* *Zero-Shot* \
  Модель выполняет задачу, опираясь только на инструкцию, без дополнительных примеров. Допустим вы можете сформулировать запрос к модели так:
  ```
  Определи, POSITIVE или NEGATIVE тон у этого текста: {текст}.
  ```
  <br>
* *Few-Shot* \
  Модель получает $k$ **примеров вместе с ответом** перед основным запросом. Это позволяет познакомить модель, например, с форматом ответа и показать какие-то паттерны через демонстрацию. \
  Рассмотрим возможный `2-shot` формат. Он предполагает демонстрацию на **двух примерах вместе с референсным ответом**. Третьим запросом будет идти целевой:

  ```
  Пример 1: "Фильм ужасен..." → NEGATIVE
  Пример 2: "Это шедевр!" → POSITIVE
  Задача: Определи тональность для "{текст}".
  ```

Обратим внимание, что Few-Shot-подход можно реализовать разными способами в зависимости от API или чат-интерфейса модели.

1. Использовать системный промпт и указать инструкцию **вместе с примерами** прямо там.

```
system_prompt = """
Ты классифицируешь тональность текста. Вот примеры:  
- "Фильм ужасен..."\nОтвет: NEGATIVE  
- "Это шедевр!"\nОтвет: POSITIVE  
Отвечай только метками POSITIVE/NEGATIVE.  
"""
```

В таком случае в `user`-реплике будет содержаться только текст целевого отзыва. При этом можно поступить иначе и использовать, например, стандартный системный промпт, рекомендованный разработчиками модели. Тогда инструкцию и few-shot примеры можно поместить сразу в `user`-реплику.


2. Используя чат-шаблон (chat-template). Тогда k-shot вариант можно реализовать следующим образом:

```
{"role": "system", "content": "Ты классифицируешь тональность текста."},  # Общая инструкция в сис. промпте, но можно и в user-реплику!
{"role": "user", "content": "Отзыв: 'Сюжет скучный...'\nОтвет:"},         # Пример 1
{"role": "assistant", "content": "NEGATIVE"},                             # Важно: ответ модели заполнен нами самостоятельно! Его можно взять из train-сета
...
{"role": "user", "content": "Отзыв: 'Лучший фильм года!'\nОтвет:"},       # Пример k
{"role": "assistant", "content": "POSITIVE"},                             # Ответ для примера k
{"role": "user", "content": "Отзыв: '{review}'\nОтвет:"}                  # Тут расположен целевой пример для прогона
```
В данном случае $k$ примеров, которые можно взять из обучающей выборки, вместе с ответами и примером для инференса принудительно помещены в историю диалога. Модель, генерируя след. реплику, выдаст нам ответ на задачу, а примеры в контексте позволят лучше следовать формату ответа и потенциально повысят итоговое качество.

Выбор стратегии, вообще говоря, зависит от пользователя и ограничений модели. Например, некоторые модели могут не поддерживать системный промпт.

---

Итак, вам требуется применить **две** open-source LLM для задачи sentiment analysis на датасете IMDB и сравнить их производительность в режимах zero- и few-shot на **валидационной подвыборке размера 2k сэмплов.**

Для few-shot используйте $k=5$ примеров, которые возьмите случайным образом из обучающей выборки. Выбор конкретного формата из описанных выше остается на вашей стороне :)


**Подумайте**, должны ли few-shot примеры быть сбалансированы, то есть содержать приблизительно одинаковое кол-во позитивных и негативных примеров? Какие стратегии выбора фьюшот примеров вы можете предложить?

В качестве LLM используйте `Qwen/Qwen2.5-3B` и `unsloth/Llama-3.2-1B-Instruct` с HuggingFace. Попробуйте **поэкспериментировать с различными вариантами промпта**. Можете взять за основу пример с семинара и воспользоваться ИИ-инструментами для его усовершенствования. Сравните результат с моделью на базе RNN, с которой вы имели дело ранее.

Представьте результат в виде аккуратной таблицы, в которой укажите модель вместе с используемым форматом (zero-/few-shot), а также итоговую точность (accuracy). Сделайте выводы.

*Замечание* \
*Для инференса рекомендуется использовать `vLLM.` Метод `.chat` может обрабатывать батч запросов, что позволит ускорить вычисления и повысить утилизацию GPU.*
```
batch_indices = range(batch_start, batch_start + batch_size)
batch_data = data.select(batch_indices)
conversations = [[{"role": "user", "content": format_text_request(review)}] for review in batch_data["review"]]
outputs = model.chat(conversations, ...)
```

In [ ]:
data = load_dataset(
    "scikit-learn/imdb", split="train"
).train_test_split(test_size=0.2, seed=42)
data

In [ ]:
validation_subsample = ...

In [ ]:
...

**Вывод**

In [ ]:
...

---
### Задача 2.

В этой задаче вы разберете пример Supervised Fine-Tuning (SFT) для новой русскоязычной базовой модели, недавно выпущенной Яндексом. Подробнее о модели можно прочесть [здесь](https://habr.com/ru/companies/yandex/articles/895428/) и [здесь](https://habr.com/ru/companies/yandex/articles/885218/).

SFT (Supervised Fine-Tuning) в данном случае — это этап дообучения предварительно обученной языковой модели на большом наборе инструктивных данных, называемый также Instruction Finetuning. Мы будем учить LLM следовать указаниям пользователя, что критически важно для создания современных ассистентов и чат-ботов. На этапе SFT модель адптируется к конкретным задачам: генеративные ответы на вопросы, диалог с пользователям, задачи классификации и генерации кода, и т. д.


In [ ]:
MODEL_NAME = "yandex/YandexGPT-5-Lite-8B-pretrain"
# Будем работать с небольшим контекстом в 2k токенов
# Современные модели обычно имеют контекст в 32k-128k токенов
# Некоторые техники позволяют добиться контекста ~1m токенов.
MAX_SEQ_LENGTH = 2048

Для начала загрузим датасет. Это смешанный набор инструктивных данных из разных источников с полезной мета-информацией. На этом датасете обучаются в том числе модели opensource-проекта [Сайга](t.me/saiga_igusev_bot).

In [ ]:
dataset = load_dataset("IlyaGusev/saiga_scored")

Датасет достаточно большого размера, содержит информацию об источнике, языке диалога, а также полезные признаки в виде оценки тематики / сложности сэмпла, полученные с помощью более производительных LLM [(Sonnet, Opus)](https://habr.com/ru/companies/bothub/articles/823580/).

In [ ]:
dataset

Посмотрим на какой-нибудь пример.

In [ ]:
dataset["train"][1800]

Вам предлагается поработать с подвыборкой из этого сета. Для этого проведите базовую аналитку: посмотрите на распр. по языкам, тематикам и сложности данных. Посчитайте базовые статистики (среднее, минимум и максимум) для реплик модели / длины диалога.
Учтите, что в большей степени нас интересуют инструкции и ответы на русском языке, небольшую аналитику можно провести для него отдельно. Не забывайте про аккуратность графиков.

In [ ]:
...

Исходя из проведенной аналитики попробуйте через различные эвристики отфильтровать датасет. При этом для простоты решения задачи важно будет оставить только одношаговые диалоги. В итоге будет достаточно 2k-3k качественных сэмплов.

In [ ]:
filtered_data = dataset["train"].filter(...)

Как вы могли заметить вместо привычного `assistant` в списке сообщений `messages` встречается роль `bot`. Давайте использовать более классический вариант.

In [ ]:
def rename_bot_in_messages(example):
    ...
    return example

filtered_data = filtered_data.map(rename_bot_in_messages)

Теперь разберемся с токенизатором.

In [ ]:
tokenizer = LlamaTokenizerFast.from_pretrained(MODEL_NAME, legacy=False)
tokenizer

С 128000 по 129024 токен находятся зарезервированные спец. токены. Они предварительно добавлены в токенизатор и эмбеддинг слой модели для удобства, но не участвовали в претрейне. Например спец. токены пригодятся нам, чтобы грамотно обучить модель следовать чат-шаблону. Можно использовать токены с 128000 включительно, поменяв представление `[SPEC_TOKEN_{IDX}]` из токенизатора на наболее привычный `<|start_header_id|>` и т. п.

**Важно:** Мы будем делать LoRA-обучение, исходные эмбеддинги трогать не будем, поэтому в случае добавления спец. токенов они останутся необучаемыми, что будет не совсем корректно. Поэтому в данном случае мы все так же будем использовать условный `<|start_header_id|>`, но токенизироваться он будет не в один спец. токен, а в несколько простых.

Будем использовать chat-template от серии моделей `llama-3`.

In [ ]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3",
    # system_message= есть возможность задать системное сообщение
)

У нашего токенизатора появился чат-шаблон! Посмотрим на него. 

In [ ]:
tokenizer.chat_template

Еще раз обратим внимание, `<|start_header_id|>` и похожие токены участвуют в шаблоне, но они НЕ добавлены как спец. токены, поэтому токенизируются в несколько токенов, как обычный текст.

In [ ]:
tokenizer("<|start_header_id|>assistant<|end_header_id|>\n\n")

Посмотрим на пример применения чат-шаблона. Обучать модель будем предсказывать только `assistant`-реплики модели.

In [ ]:
chat_template_sample = tokenizer.apply_chat_template(filtered_data[1800]["messages"], tokenize=False)
chat_template_sample

Теперь загрузим модель

In [ ]:
# Будем использовать квантизацию в 4бита, подробнее, что это значит
# мы узнаем на след. занятии
dtype = None
load_in_4bit = True

# Загрузка модели через знакомый метод .from_pretrained
model, unsloth_tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

Важно помнить, что это пока базовая модель, она просто продолжает текст!

In [ ]:
prompt = "План изучения машинного обучения:"
device = torch.device("cuda:0")
tokenized_prompt = tokenizer(prompt, return_tensors="pt").to(device)
# Посмотрим на генерацию
outputs = model.generate(**tokenized_prompt,
                         do_sample=True,
                         temperature=0.3,
                         max_new_tokens=128,
                         # Используем KV-cache, а не пересчитываем векторы key/value
                         use_cache=True)
tokenizer.batch_decode(outputs)[0]

Теперь добавим обучаемые LoRA-адаптеры. Полноценный SFT в рамках Colab'a будет сделать сложно, поэтому обойдемся peft-методом, функционал есть прямо в `unsloth`!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    # Знакомые гиперпараметры LoRA
    r=...,
    target_modules=[
        "q_proj",    # Attention
        "k_proj",    # Attention
        "v_proj",    # Attention
        "o_proj",    # Attention
        "gate_proj", # MLP
        "up_proj",   # MLP
        "down_proj", # MLP
    ],
    lora_alpha=...,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth", # Подробнее на след. занятии
    random_state=3407,
    use_rslora=False,  # Без rank stabilized LoRA
    loftq_config=None, # And LoftQ
)

Напишем функцию для применения чат-шаблона к каждому сэмплу. Учтите, что токенизировать текст на этом этапе не стоит, добавлять спец. токены и `generation_prompt` тоже. Важно грамотно указать аргументы `.apply_chat_template(...)`.

In [ ]:
def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = ...
    return {
        # Отформатированные тексты сообщений для последующей токенизации
        "text" : texts
    }

filtered_data = filtered_data.map(formatting_prompts_func, batched=True)

Посмотрим на получившийся пример. В поле `text` должно быть что-то похожее:
```
<s><|start_header_id|>user<|end_header_id|>\n\nкрасивые места ульяновска<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nУльяновск — город ....<|eot_id|>
```

In [ ]:
filtered_data[5]["text"]

Теперь зададим data-collator. Этот объект подготавливает тензоры перед форвародом. Обратите внмиание, мы будем использовать `DataCollatorForCompletionOnlyLM`. Он позволяет обучаться только ответах модели, то есть на токенах после `assistant<|end_header_id|>\n\n`. 

In [ ]:
tokenizer.add_bos_token = False # при токенизации не добавляем bos, т.к. он есть в чат-шаблоне и уже находится в сэмплах.
# Предварительно токенизированная часть  assistant<|end_header_id|>\n\n
tokenized_assistant_response_template = [108651, 125906, 125853, 617, 125865, 8264, 125865, 367, 73136, 3, 3]
# Обучаться будем только на ответах модели, то есть токенах после assistant<|end_header_id|>\n\n
collator = DataCollatorForCompletionOnlyLM(
    # явно указываем начало assistant-реплики, до нее включительно будет стоять -100 для подсчет Cross-Entropy
    response_template=tokenized_assistant_response_template,
    tokenizer=tokenizer
)

Зададим трейнер.

In [ ]:
trainer = SFTTrainer(
    model=...,
    tokenizer=...,
    train_dataset=...,
    # Передаем поле, в котором отформатированные сообщения
    # Токенизацию выполнит trainer
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGT,
    dataset_num_proc=2,
    data_collator=collator,
    packing=False,
    args = TrainingArguments(
        per_device_train_batch_size=2,
        # На след. занятии поговорим об этом
        gradient_accumulation_steps=4,
        # Используем небольшой warmup
        warmup_steps=5,
        # Для знакомства модели с SFT форматом будет достаточно
        max_steps=100,
        learning_rate=2e-4,
        # На след. занятии поговорим об этом
        fp16=True,
        logging_steps=1,
        # На след. занятии поговорим об этом
        optim="adamw_8bit",
        # Регуляризация
        weight_decay=0.01,
        # Линейный шедулер
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none"
    ),
)

Посмотрим, как работает `DataCollatorForCompletionOnlyLM`. Обратите внимание, что в `labels` имеется некоторый префикс из `-100`. Так мы обучаем модель предсказывать токены реплики модели, игнорируя при подсчете лосса запрос юзера и системный промпт. При этом, в наших данных сейчас только одношаговые диалоги. В более продвинутом случае (многошаговые диалоги) процесс маскирования реплик модели был бы немного сложнее.

In [ ]:
collator([trainer.train_dataset[7]["input_ids"]])

Обучим модель. Если вы нигде значительно не ошиблись, то лосс будет в районе 0.7 – 1.4.

In [ ]:
trainer_stats = trainer.train()

Настало время проверить модель на разных запросах. Сделать это можно так:

In [ ]:
# Перевод модели в eval-model
FastLanguageModel.for_inference(model)

# Уже знакомый код генерации
messages = [
    {"role": "user", "content": "Расскажи коротко, что такое лог регрессия?"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=128,
    use_cache=True, # Использование KV-cache, а не пересчет
    do_sample=True,
    temperature=0.6
)
tokenizer.batch_decode(outputs)[0]

Если вы все сделали корректно, то модель точно должны выучить SFT-формат и в целом ответить на запрос, при чем достаточно развернуто.

Обучите еще один вариант модели, достаточно отличающийся от начального. Например, вы можете использовать другие эвристики для фильтрации датасета или кардинально, но разумно изменить гиперпараметры обучения или LoRA-адаптеров.
Сравните две получившиеся модели на 5-10 содержательных запросах, оцените результаты и сделайте выводы.

In [ ]:
...